# <a id='toc1_'></a>[set up](#toc0_)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
if sys.platform == 'linux':
    sys.path.append("/home/qix/MultiNeuronGLM")
else:
    sys.path.append("D:/Github/MultiNeuronGLM")

In [2]:
import pandas as pd
import utility_functions as utils
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import random
import torch
import logging
import joblib

import GLM
from DataLoader import Allen_dataset, Allen_dataloader_multi_session

# sns.set_theme()
sns.set_theme(style="white")
# sns.set_style('whitegrid')

/home/qix/anaconda3/envs/allen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set random seed for reproducibility

random.seed(0)
np.random.seed(0) 
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)

logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s - [%(filename)s:%(lineno)d]'
)

# Load all sessions

In [6]:
### Build a dataloader for cross-session data
### Only two sessions
# session_ids = [757216464, 757970808]

### Session with all 6 probes
session_ids = [
    715093703, 719161530, 721123822, 737581020, 739448407, 742951821, 743475441,
    744228101, 746083955, 750332458, 750749662, 751348571, 754312389, 755434585,
    756029989, 757216464, 760693773, 761418226, 762602078, 763673393, 766640955,
    767871931, 768515987, 771160300, 771990200, 773418906, 774875821, 778240327,
    778998620, 779839471, 781842082, 786091066, 787025148, 789848216, 791319847,
    793224716, 794812542, 797828357, 798911424, 799864342, 831882777, 839068429,
    840012044, 847657808
]

### All sessions
# session_ids = [
#     715093703, 719161530, 721123822, 732592105, 737581020, 739448407,
#     742951821, 743475441, 744228101, 746083955, 750332458, 750749662,
#     751348571, 754312389, 754829445, 755434585, 756029989, 757216464,
#     757970808, 758798717, 759883607, 760345702, 760693773, 761418226,
#     762120172, 762602078, 763673393, 766640955, 767871931, 768515987,
#     771160300, 771990200, 773418906, 774875821, 778240327, 778998620,
#     779839471, 781842082, 786091066, 787025148, 789848216, 791319847,
#     793224716, 794812542, 797828357, 798911424, 799864342, 816200189,
#     819186360, 819701982, 821695405, 829720705, 831882777, 835479236,
#     839068429, 839557629, 840012044, 847657808
# ]

kwargs = {
    'shuffle':False,
    'align_stimulus_onset':True,
    'merge_trials':True,
    'batch_size':64,
    'fps':500,
    'start_time':0.0,
    'end_time':0.4,
    'padding':0.1,
    'selected_probes':['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF'],
}
cross_session_dataloader = Allen_dataloader_multi_session(session_ids, **kwargs)

2025-05-02 06:51:23,032 - CRITICAL - Start loading data - [DataLoader.py:171]
100%|██████████| 44/44 [26:51<00:00, 36.63s/it]


Total sessions: 44, Batch size: 64, Train set size: 5073, Val set size: 754, Test set size: 1485


In [7]:
# Save the object
# data_path = '/home/qix/user_data/allen_spike_trains/two_sessions.joblib'
data_path = '/home/qix/user_data/allen_spike_trains/all_six_probessessions.joblib'
joblib.dump(cross_session_dataloader, data_path)

['/home/qix/user_data/allen_spike_trains/all_six_probessessions.joblib']

In [7]:
# Load the object
# data_path = '/home/qix/user_data/allen_spike_trains/two_sessions.joblib'
data_path = '/home/qix/user_data/allen_spike_trains/all_six_probessessions.joblib'
cross_session_dataloader = joblib.load(data_path)